In [1]:
import requests
import xml.etree.ElementTree as ET
import time
from collections import Counter
from itertools import chain
import re

In [2]:
with open('data/generated_texts.txt') as f:
    lines = f.read()
generated_data=[]
for line in lines.split("="*40 + "\n"):
    if len(line.replace("\n",""))!=0:
        generated_data.append(line.replace("\n",""))
generated_data={(i+1): generated_data[i] for i in range(0,len(generated_data))}


from simpletransformers.language_representation import RepresentationModel
gpt2_model = RepresentationModel(
        model_type="gpt2",
        model_name="gpt2-medium",
        use_cuda=False
    )
generated_vectors = gpt2_model.encode_sentences(list(generated_data.values()), combine_strategy='mean')

from scipy.spatial import distance
def similarity(doc, topicId):
    doc_vector = gpt2_model.encode_sentences([doc], combine_strategy='mean')
    similarity_score = 1- distance.cosine(doc_vector,generated_vectors[topicId-1])
    return similarity_score

Some weights of the model checkpoint at gpt2-medium were not used when initializing GPT2ForTextRepresentation: ['wte.weight', 'wpe.weight', 'h.0.ln_1.weight', 'h.0.ln_1.bias', 'h.0.attn.bias', 'h.0.attn.c_attn.weight', 'h.0.attn.c_attn.bias', 'h.0.attn.c_proj.weight', 'h.0.attn.c_proj.bias', 'h.0.ln_2.weight', 'h.0.ln_2.bias', 'h.0.mlp.c_fc.weight', 'h.0.mlp.c_fc.bias', 'h.0.mlp.c_proj.weight', 'h.0.mlp.c_proj.bias', 'h.1.ln_1.weight', 'h.1.ln_1.bias', 'h.1.attn.bias', 'h.1.attn.c_attn.weight', 'h.1.attn.c_attn.bias', 'h.1.attn.c_proj.weight', 'h.1.attn.c_proj.bias', 'h.1.ln_2.weight', 'h.1.ln_2.bias', 'h.1.mlp.c_fc.weight', 'h.1.mlp.c_fc.bias', 'h.1.mlp.c_proj.weight', 'h.1.mlp.c_proj.bias', 'h.2.ln_1.weight', 'h.2.ln_1.bias', 'h.2.attn.bias', 'h.2.attn.c_attn.weight', 'h.2.attn.c_attn.bias', 'h.2.attn.c_proj.weight', 'h.2.attn.c_proj.bias', 'h.2.ln_2.weight', 'h.2.ln_2.bias', 'h.2.mlp.c_fc.weight', 'h.2.mlp.c_fc.bias', 'h.2.mlp.c_proj.weight', 'h.2.mlp.c_proj.bias', 'h.3.ln_1.weight'

In [4]:
generated_data[10]

"The first thing you need to do is to find a comfortable bed.It's not always easy to find a comfortable bed, but it's always possible.You can find a bed in a small room, in a room with a window, or in a room with a window.You can find a bed in a room with a window, in a room with a window. You can find a bed in a room with a window, in a room with a"

In [ ]:
def get_titles(file):
    tree = ET.parse(file)
    root = tree.getroot()
    buffer = []
    i=1
    for title in root.iter('title'):
        if i<=int(n_topics):
            buffer.append(title.text.strip())
        i=i+1
    return buffer
def remove_htmlTags(doc):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', doc)
def get_texts_title_snippet(trecIds_titles_snippets):
    docs={}
    for trec_id, title, snippet in trecIds_titles_snippets:
        docs[trec_id] = title + ". " + snippet
    return docs

In [ ]:
import statistics
def update_similarity_pagerank(similarity_scores,resp, statusAdd):
    '''
    spam rank high -> more spam
    '''
    new_resp = []
    if statusAdd==False: #using directly similarity score for reranking
        for doc in resp['results']:
            trec_id = doc['trec_id']
            spam_rank=float()
            
            if str(doc['spam_rank'])=="null":
                spam_rank = statistics.mean([doc['spam_rank'] for doc in resp['results'] if str(doc['spam_rank'])!="null" ])
            else:
                spam_rank=float(doc['spam_rank'])
            similarity_score= similarity_scores[trec_id]
            
            #change end_score
            doc['score'] = similarity_score/spam_rank
            new_resp.append(doc)
    else:
        for doc in resp['results']:
            trec_id = doc['trec_id']
            
            spam_rank=float()
            if str(doc['spam_rank'])=="null":
                spam_rank = statistics.mean([doc['spam_rank'] for doc in resp['results'] if str(doc['spam_rank'])!="null" ])
            else:
                spam_rank=float(doc['spam_rank'])
            
            relevance_score=float(doc['score'])
            similarity_score= similarity_scores[trec_id]
            
            #change end_score
            doc['score'] = relevance_score*(1+similarity_score) - spam_rank
            new_resp.append(doc)
            
    return {'results':sorted(new_resp,key= lambda doc: doc['score'], reverse=True)}
def update_response(similarity_scores,resp, statusAdd):
    new_resp = []
    if statusAdd==False: #using directly similarity score for reranking
        for doc in resp['results']:
            trec_id = doc['trec_id']
            #relevance_score=float(doc['score'])
            similarity_score= similarity_scores[trec_id]
            
            #change end_score
            doc['score'] = similarity_score
            new_resp.append(doc)
    else:
        for doc in resp['results']:
            trec_id = doc['trec_id']
            relevance_score=float(doc['score'])
            similarity_score= similarity_scores[trec_id]
            
            #change end_score
            doc['score'] = relevance_score*(1+similarity_score)
            new_resp.append(doc)
            
    return {'results':sorted(new_resp,key= lambda doc: doc['score'], reverse=True)}

In [ ]:
def chatnoir_api(topicID, topic, size, spamRank, statusAdd):
    
    url = 'https://www.chatnoir.eu/api/v1/_search'
    request_data = {
        "apikey": "67fac2d9-0f98-4c19-aab0-18c848bfa130",
        "query": topic,
        "size": size,
        "index": ["cw12"],
    }
    def chatnoir_req():
        try:
            response_=requests.post(url, data=request_data)
            response_.raise_for_status()
            return response_.json()
        except requests.exceptions.HTTPError:
            time.sleep(1)
            return chatnoir_req()    
    resp = chatnoir_req()
    print("="*40 + "GET RESPONSES" + "="*40)
    results = resp['results']
    trecIds_titles_snippets = [(results[i]['trec_id'],remove_htmlTags(results[i]['title']), remove_htmlTags(results[i]['snippet'])) for i in range(0,len(results))]
    docs= get_texts_title_snippet(trecIds_titles_snippets) #only titles and snippets
    
    similarity_scores={}
    for trec_id, doc in docs.items():
        print(remove_htmlTags(doc))
        score = similarity(doc, topicID)
        print(score)
        similarity_scores[trec_id]=score
    #print(similarity_scores)
    
    #similarity_scores = dict(sorted(x.items(), key=lambda item: item[1]))
    if spamRank==False:
        resp=update_response(similarity_scores,resp, statusAdd)
    else:
        resp=update_similarity_pagerank(similarity_scores,resp, statusAdd)
    #print(resp)
    return similarity_scores, resp

In [ ]:
def main(spamRank, statusAdd):
    topics = get_titles(file)
    print(topics[0])
    out = open("data/output_similarity_"+str(spamRank)+"_"+str(statusAdd)+".run", "w")
    answers = []
    
    index=1 #TOPIC ID
    for topic in topics:
        print("Getting response for", topic)
        answers.append(chatnoir_api(index, topic, size, spamRank, statusAdd)[1])
        #if(index==1):
        #    break
        index=index+1
        
    topicId = 1
    for topic in answers:
        print(topic)
        rank = 1
        for response in topic['results']:
            buffer = topicId, "Q0", response['trec_id'], rank, response['score'], "JackSparrowVanilla"
            out.write(" ".join(map(str, buffer)) + "\n")
            rank += 1
        topicId += 1

In [ ]:
n_topics=50
file="data/topics-task-2.xml"
statusAdd=False
spamRank=True#using only similarity for reranking
size=15
main(spamRank, statusAdd)

In [ ]:
run compare_with_baseline.py data/touche2020-task2-relevance-withbaseline.qrel data/base_chatnoir.run data/output_similarity_False_True.run 'ndcg_cut_5'

In [ ]:
n_topics=50
file="data/topics-task-2.xml"
statusAdd=True
spamRank=True#using similarity score to update relevance score
size=15
main(spamRank, statusAdd)

In [ ]:
run compare_with_baseline.py data/touche2020-task2-relevance-withbaseline.qrel data/base_chatnoir.run data/output_similarity_True_True.run 'ndcg_cut_5'

In [ ]:
def specific_topic(index, size, spamRank, statusAdd):
    topics = get_titles(file)
    specific_topic=topics[index-1]
    print(specific_topic)
    print("GENERATED TEXT: " + generated_data[index])
    similarity_scores, resp = chatnoir_api(index, specific_topic, size, spamRank, statusAdd)
    print(similarity_scores)
    return similarity_scores, resp
n_topics=50
file="data/topics-task-2.xml"

similarity_scores, resp = specific_topic(36, 15,True, True)
statistics.mean(similarity_scores.values())